[For converting torch to tensorrt](https://github.com/NVIDIA-AI-IOT/torch2trt)

This file will run in google collab

In [10]:
!pip install --upgrade --index-url https://pypi.ngc.nvidia.com nvidia-tensorrt
!git clone https://github.com/NVIDIA-AI-IOT/torch2trt
%cd torch2trt
!python setup.py install


Looking in indexes: https://pypi.ngc.nvidia.com, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'torch2trt' already exists and is not an empty directory.
/content/torch2trt
running install
running bdist_egg
running egg_info
creating torch2trt.egg-info
writing torch2trt.egg-info/PKG-INFO
writing dependency_links to torch2trt.egg-info/dependency_links.txt
writing top-level names to torch2trt.egg-info/top_level.txt
writing manifest file 'torch2trt.egg-info/SOURCES.txt'
adding license file 'LICENSE.md'
writing manifest file 'torch2trt.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/torch2trt
copying torch2trt/dataset.py -> build/lib/torch2trt
copying torch2trt/flatten_module.py -> build/lib/torch2trt
copying torch2trt/module_test.py -> build/lib/torch2trt
copying torch2trt/__init__.py -> build/lib/torch2trt
copying torch2trt/test.py -> build

Import pytorch model and convert to tensorRT

In [11]:
import torch
from torch2trt import torch2trt
from torchvision.models.alexnet import alexnet

# create some regular pytorch model...
model = alexnet(pretrained=True).eval().cuda()

# create example data
x = torch.ones((1, 3, 224, 224)).cuda()

# convert to TensorRT feeding sample data as input
model_trt = torch2trt(model, [x])

/content/torch2trt/torch2trt/dataset.py:61: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(self) > 0, 'Cannot create default flattener without input data.')
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

Use mock tensor to test ouput against pytorch model

In [12]:
y = model(x)
y_trt = model_trt(x)

# check the output against PyTorch
print(torch.max(torch.abs(y - y_trt)))

tensor(1.1921e-06, device='cuda:0', grad_fn=<MaxBackward1>)


Save tensor rt model

In [13]:
torch.save(model_trt.state_dict(), 'alexnet_trt.pth')


Load tensorRT model

In [14]:
from torch2trt import TRTModule

model_trt = TRTModule()

model_trt.load_state_dict(torch.load('alexnet_trt.pth'))

<All keys matched successfully>